In [1]:
!curl -q https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --apt-packages libomp5 libopenblas-dev
!pip -q install seqeval
!pip install transformers

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5116  100  5116    0     0  59488      0 --:--:-- --:--:-- --:--:-- 59488
Updating... This may take around 2 minutes.
Updating TPU runtime to pytorch-dev20200515 ...
     |████████████████████████████████| 61kB 3.6MB/s 
Uninstalling torch-1.7.0+cu101:
  Found existing installation: google-api-python-client 1.7.12
    Uninstalling google-api-python-client-1.7.12:
      Successfully uninstalled google-api-python-client-1.7.12
Done updating TPU runtime
  Successfully uninstalled torch-1.7.0+cu101
Uninstalling torchvision-0.8.1+cu101:
  Successfully uninstalled torchvision-0.8.1+cu101
Copying gs://tpu-pytorch/wheels/torch-nightly+20200515-cp36-cp36m-linux_x86_64.whl...
- [1 files][ 91.0 MiB/ 91.0 MiB]                                                
Operation completed over 1 objects/91.0 MiB.                                     


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch
import torch_xla
import torch_xla.core.xla_model as xm

In [4]:
import numpy as np
import pandas as pd
import transformers
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertForTokenClassification, BertTokenizer, BertConfig, BertModel

dev = xm.xla_device()

In [5]:
## Possible Tags
tag_to_idx = {'O': 0, 'T-NEG': 1, 'T-POS': 2, 'T-NEU': 3, 'PAD':4}

## Data files
train_path = '/content/drive/MyDrive/sem5/NLP/train.txt'
test_path = '/content/drive/MyDrive/sem5/NLP/test.txt'

idx = 0

def get_data(path):
  f = open(path, 'r')
  data = []
  labels = []
  m1 = -1
  for line in f:
    line = line.split('####')
    data.append(line[0])
    tags = []
    line[1] = line[1].split()
    m1 = max(m1,len(line[1]))
    for word in line[1]:
      word = word.split('=')
      tags.append(tag_to_idx[word[1]])
    labels.append(tags)
  return data, labels, m1

train_data, train_labels, m1 = get_data(train_path)
test_data, test_labels, m2 = get_data(test_path)

In [6]:
MAX_LEN = 200
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 16
EPOCHS = 5
lr = 2e-4
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [7]:
class dataset(Dataset):
    def __init__(self, tokenizer, sentences, labels, max_len):
        self.len = len(sentences)
        self.sentences = sentences
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __getitem__(self, index):
        sentence = str(self.sentences[index])
        inputs = self.tokenizer.encode_plus(
            sentence,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        label = self.labels[index]
        label.extend([4]*200)
        label=label[:200]

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'tags': torch.tensor(label, dtype=torch.long)
        } 
    
    def __len__(self):
        return self.len

In [ ]:
train_sentences = train_data
test_sentences = test_data
training_set = dataset(tokenizer, train_sentences, train_labels, MAX_LEN)
testing_set = dataset(tokenizer, test_sentences, test_labels, MAX_LEN)

In [9]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [10]:
class BERT(torch.nn.Module):
    def __init__(self):
        super(BERT, self).__init__()
        self.l1 = transformers.BertForTokenClassification.from_pretrained('bert-base-cased', num_labels=5)
    
    def forward(self, ids, mask, labels):
        output_1= self.l1(ids, mask, labels = labels)
        return output_1

In [11]:
model = BERT()
model.to(dev)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

BERT(
  (l1): BertForTokenClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(28996, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
            

In [12]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=lr)

In [13]:
def train(epoch):
    model.train()
    for _,data in enumerate(training_loader, 0):
        ids = data['ids'].to(dev, dtype = torch.long)
        mask = data['mask'].to(dev, dtype = torch.long)
        targets = data['tags'].to(dev, dtype = torch.long)
        loss = model(ids, mask, labels = targets)[0]
        if _%500==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        xm.optimizer_step(optimizer)
        xm.mark_step() 

In [14]:
for epoch in range(EPOCHS):
    train(epoch)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Epoch: 0, Loss:  1.5636248588562012
Epoch: 1, Loss:  0.3350532650947571
Epoch: 2, Loss:  0.2534438371658325
Epoch: 3, Loss:  0.12250971049070358
Epoch: 4, Loss:  0.11334336549043655


In [15]:
from seqeval.metrics import f1_score

def cal_accuracy(preds, labels):
    flat_preds = np.argmax(preds, axis=2).flatten()
    flat_labels = labels.flatten()
    return np.sum(flat_preds == flat_labels)/len(flat_labels)

In [16]:
tag_vals = ['O', 'T-NEG', 'T-POS', 'T-NEU']

def valid(model, testing_loader):
    model.eval()
    eval_loss = 0; eval_accuracy = 0
    n_correct = 0; n_wrong = 0; total = 0
    predictions , true_labels = [], []
    nb_eval_steps, nb_eval_examples = 0, 0
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            ids = data['ids'].to(dev, dtype = torch.long)
            mask = data['mask'].to(dev, dtype = torch.long)
            targets = data['tags'].to(dev, dtype = torch.long)

            output = model(ids, mask, labels=targets)
            loss, logits = output[:2]
            logits = logits.detach().cpu().numpy()
            label_ids = targets.to('cpu').numpy()
            predictions.append([list(p) for p in np.argmax(logits, axis=2)])
            true_labels.append(label_ids)
            accuracy = cal_accuracy(logits, label_ids)
            eval_loss += loss.mean().item()
            eval_accuracy += accuracy
            nb_eval_examples += ids.size(0)
            nb_eval_steps += 1
        eval_loss = eval_loss/nb_eval_steps
        print("Validation loss: {}".format(eval_loss))
        print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
        return predictions, true_labels

In [17]:
# To get the results on the validation set. This data is not seen by the model
p,t = valid(model, testing_loader)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Validation loss: 0.30905826613306997
Validation Accuracy: 0.33315625000000004


In [ ]:
y_true = []
y_pred = []
for batch in range(50):
  for entry in range(16):
    N = 0
    while t[batch][entry][N] != 4:
      N+=1
    try:
      a = [tag_vals[p[batch][entry][i]] for i in range(N)]
      y_pred.append(a)
      b = [tag_vals[t[batch][entry][i]] for i in range(N)]
      y_true.append(b)
    except:
      print(batch, entry)

In [19]:
print("F1-Score: {}".format(f1_score(y_pred, y_true)))

F1-Score: 0.45916114790286977


/usr/local/lib/python3.6/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: T-POS seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.6/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: T-NEU seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.6/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: T-NEG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


In [20]:
for i in range(5):
  print('Expected:', y_true[i])
  print('Output:', y_pred[i])

Expected: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Output: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Expected: ['O', 'T-POS', 'T-POS', 'T-POS', 'O', 'O', 'O', 'O', 'O', 'T-POS', 'O', 'O', 'O', 'T-POS', 'O']
Output: ['O', 'O', 'T-POS', 'T-POS', 'O', 'O', 'O', 'O', 'T-POS', 'O', 'O', 'O', 'T-POS', 'T-POS', 'O']
Expected: ['O', 'O', 'O', 'T-NEU', 'O', 'T-NEU', 'O', 'O', 'O']
Output: ['O', 'O', 'T-NEU', 'T-POS', 'T-POS', 'O', 'O', 'O', 'O']
Expected: ['O', 'O', 'O', 'O', 'O', 'T-NEG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Output: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'T-NEG', 'T-NEG']
Expected: ['O', 'T-NEG', 'O', 'O', 'O', 'O', 'O', 'O', 'T-NEG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'T-NEU', 'O', 'O', 'O']
Output: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'T-NEG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [22]:
PATH = '/content/drive/MyDrive/sem5/NLP/absa.sav'
## Save
torch.save(model.state_dict(), PATH)
## Load
# model = BERT()
# model.load_state_dict(torch.load(PATH))
# model.eval()